# Import packages

In [107]:
from pyntcloud import PyntCloud
import math
from PIL import Image
import glob 
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
from IPython.display import display
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Initialize some parameters

In [108]:
focalLength = 365.2946 # The focalLength is a specific characteristic of the camera.
centerX=259.7606   # The values of centerX and centerY are obtained from intrinsic camera calibration.
centerY=205.8992
scalingFactor = 100.0# It is equal to 1000 because the depth is in milimeter.             

In [109]:
def cloud(frame,frame2,width,height):
    # This function returns a 3D points cloud from an RGB-D frame
    
    count=0
    points=[]
    for u in range(width):
        for v in range(height):
            color = frame2[u,v]
            Z = frame[u,v]/ scalingFactor
            if Z==0: continue
            X = (v - centerX) * Z / focalLength
            Y = (u - centerY) * Z / focalLength
            points.append([X,Y,Z,np.uint(color[0]),np.uint(color[1]),np.uint(color[2])])
            count+=1
    points=np.array(points)
    points=pd.DataFrame(points,columns=['x', 'y','z',"red","green","blue"])
    points["red"]=points["red"].astype('uint8')
    points["green"]=points["green"].astype('uint8')
    points["blue"]=points["blue"].astype('uint8')
    mycloud=PyntCloud(points)
    return mycloud    

In [110]:
def medfilter(frame1):
    # this function removes pepper noise
    fig = plt.figure()
    image_gray=frame1
    res= ndimage.median_filter(frame1, size=5)
    return res

In [111]:
def admissible_region(SMAR,DRG,TDBG,current_cloud):
    # This function eliminates the uselsess regions for the robot(what behind the rebot,...)
    admi_region=current_cloud.points.loc[(current_cloud.points['y'] >= TDBG )]
    admi_region=pd.DataFrame(admi_region,columns=['x', 'y','z',"red","green","blue"])
    admi_region=admi_region.loc[(admi_region['y'] <= (SMAR+ DRG))]
    admi_region=pd.DataFrame(admi_region,columns=['x', 'y','z',"red","green","blue"])
    admi_region=admi_region.loc[(admi_region['z'] <= 4)]
    admi_region=pd.DataFrame(admi_region,columns=['x', 'y','z',"red","green","blue"])
    admi_region=admi_region.loc[(admi_region['z'] >= 0)]
    admi_region=pd.DataFrame(admi_region,columns=['x', 'y','z',"red","green","blue"])
    mycloud1=PyntCloud(admi_region)
    return mycloud1

In [112]:
def videoread(path1,path2):
     cap = cv2.VideoCapture(path1)
     cap1= cv2.VideoCapture(path2)
     fps = int(cap.get(5))# number of frames per second      # 'Depth.mp4'
     width  = int(cap.get(3))# width of a frame
     height = int(cap.get(4))# height of a frame
     fps1 = int(cap1.get(5))# number of frames per second      # 'Depth.mp4'
     width1  = int(cap1.get(3))# width of a frame
     height1 = int(cap1.get(4))# height of a frame
     a1=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # number of frames in the video
     a2=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
     return cap,cap1, a1,a2,width,height,fps

In [113]:
def voxelgridfilter(current_cloud):
    # This filter aims to remove outlier measurements: representing each 3D box by its centroid.
    # This filter aims to reduce the computational complexity of the code.
    voxelgrid_id = current_cloud.add_structure("voxelgrid", size_x=0.025, size_y=0.025, size_z=0.025)
    new_cloud = current_cloud.get_sample("voxelgrid_centroids",voxelgrid_id=voxelgrid_id)
    new_cloud1=PyntCloud(new_cloud)
    return new_cloud1

In [114]:
def normalvectors(n,cloud):
    # This function gives the normal vector of each point that belongs to the cloud.
    neighbors=cloud.get_neighbors(k=n)
    cloud.add_scalar_field("normals", k_neighbors=neighbors)
    return cloud

In [115]:
def Determine_obs(N,θ,N1,cloud,number,grad):
    # This function returns the point cloud of the obstacles in the scence.
    # N is the minimum number of neighbors
    # R is the radius of spherical neighborhood
    # θ is the maximum angle of passable slopes
    cs=math.cos(θ)# threshold
    obs=cloud.points.loc[(abs(cloud.points.iloc[ :, 4])  < cs)]# determine obstacle points
    safe=cloud.points.loc[(abs(cloud.points.iloc[ :, 4]) >= cs)]#determine non-obstacle points
    new_list_safe=safe.values.tolist()
    new_df_negative=[] #non obstacle pixels
    for pi_safe in range(len(new_list_safe)):
        s_safe=np.array(new_list_safe[pi_safe])
        v_safe=int(round(focalLength*list(s_safe)[0]/list(s_safe)[2]+centerX,2))
        u_safe=int(round(focalLength*list(s_safe)[1]/list(s_safe)[2]+centerY,2))
        if u_safe< grad.shape[0] and v_safe< grad.shape[1]:
              l_safe=[number,u_safe,v_safe,new_list_safe[pi_safe][4],0,grad[u_safe,v_safe],0]
              new_df_negative.append(l_safe)
    
    new_cloud2=PyntCloud(obs)
    neighall=cloud.get_neighbors(k=7)
    lindex=[]
    for i in range(cloud.points.shape[0]):
            if (abs(cloud.points.iloc[i, 4]) < cs):
                lindex.append(neighall[i])
    new_list1=cloud.points.values.tolist()
    new_list=obs.values.tolist()
    final_points=[]
    new_df_positive=[]#to save obstacle pixels
    for pi in range(len(new_list)):
        s=np.array(new_list[pi][:3])
        n=0
        for pj in (list(lindex[pi])):
                if  (new_list1[pj][4]<cs) :
                        n=n+1  
        if(n> int(N1/2)):
                v=int(round(focalLength*list(s)[0]/list(s)[2]+centerX,2))
                u=int(round(focalLength*list(s)[1]/list(s)[2]+centerY,2))
                l=[number,u,v,new_list[pi][4],1,grad[u,v],1]
                new_df_positive.append(l)
                final_points.append(s)
    new_df_positive=np.array(new_df_positive)
    new_df_positive=pd.DataFrame(new_df_positive,columns=['frame','u', 'v','slope','y','grad','e'])
    new_df_negative=np.array(new_df_negative)
    new_df_negative=pd.DataFrame(new_df_negative,columns=['frame','u', 'v','slope','y','grad','e'])
    
    return final_points,new_df_positive,new_df_negative # new_df_positive contains the information of obstacle points, new_df_negative contains the information of safe pixels 

In [116]:
def result(final_points,color,width,height):
    # This function represents the obtained points that correspond to obstacles in the scene.
    v=[int(round(focalLength*cr[0]/cr[2]+centerX,2)) for cr in  final_points ]
    u=[int(round(focalLength*cr[1]/cr[2]+centerY,2)) for cr in final_points]
    
    for (i,j) in zip (u,v):
                color[i,j]= (255,0,0)
                
                
                
            
    return (color)

In [117]:
def main(n,θ,N,N1,SMAR,DRG,TDBG,path,path1):
    # This is the main function that contains all the operations.
    # This function returns the safe points and the obstacle points of each frame
    cap,cap1,a,a1,width,height,fps= videoread(path,path1)
    listofresultedframes=[]
    list_of_df_negatives=[]
    list_of_df_positives=[]
    list_of_grad=[]
    for k in range(10):
            ret, frame = cap.read()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            (w, h)= gray.shape
            ret1, frame1=cap1.read()
            rgb = cv2.resize(frame1,(h, w))
            #framevisualization(gray,rgb)
            gray=medfilter(gray)
            window_name = ('Sobel Edge Detector')
            scale = 1
            delta = 0
            ddepth = cv2.CV_16S
            grad_x = cv2.Sobel(gray, ddepth,1, 0, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
            grad_y = cv2.Sobel(gray, ddepth, 0, 1, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
            abs_grad_x = cv2.convertScaleAbs(grad_x)
            abs_grad_y = cv2.convertScaleAbs(grad_y)
            grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
            if ret == True:
                mycloud=cloud(gray,rgb,w,h)
                mycloud1=admissible_region(SMAR,DRG,TDBG,mycloud)
                mycloud2=voxelgridfilter(mycloud1)
                mycloud3=normalvectors(n,mycloud2)
                final_points,new_df_positive,new_df_negative=Determine_obs(N,θ,N1,mycloud3,k,grad)
                list_of_df_positives.append(new_df_positive)
                list_of_df_negatives.append(new_df_negative)
                b = np.isnan(np.array(final_points))
                img=result(final_points,rgb,w,h)
                listofresultedframes.append(img)
    k=listofresultedframes
    return(k,fps,width,height,list_of_df_positives,list_of_df_negatives)

In [118]:
k,fps,width,height,list_of_df_positives,list_of_df_negatives= main(5,math.pi / 3,3,7,0.25,0.35,-0.3,"Depth1.mp4","RGB1.mp4")

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [119]:
#get the data of posistive examples: obstacle points
df_positive = pd.concat(list_of_df_positives)

In [120]:
df_positive.head()

,frame,u,v,slope,y,grad,e
0,0.0,67.0,2.0,0.238325,1.0,4.0,1.0
1,0.0,76.0,2.0,-0.053750,1.0,4.0,1.0
2,0.0,89.0,3.0,-0.071315,1.0,4.0,1.0
3,0.0,101.0,3.0,-0.072098,1.0,4.0,1.0
4,0.0,115.0,4.0,-0.045022,1.0,2.0,1.0


In [121]:
#get the data of negative examples: safe points
df_negative = pd.concat(list_of_df_negatives)

In [122]:
df_negative.shape

(11274, 7)

In [123]:
df_negative.head()

,frame,u,v,slope,y,grad,e
0,0.0,343.0,0.0,0.538736,0.0,0.0,0.0
1,0.0,328.0,14.0,0.547290,0.0,4.0,0.0
2,0.0,347.0,0.0,0.924051,0.0,1.0,0.0
3,0.0,354.0,2.0,0.745034,0.0,5.0,0.0
4,0.0,351.0,3.0,-0.732903,0.0,2.0,0.0


In [124]:
print(df_positive.shape)
print(df_negative.shape)

(17122, 7)
(11274, 7)


In [125]:
l_data=[df_positive,df_negative]
data = pd.concat(l_data)
#get the training data by merging data from the two classes, negative and positive

In [126]:

data.shape

(28396, 7)

In [127]:
e1=[0 for i in range(data.shape[0])]
e2=[0 for i in range(data.shape[0])]
e3=[0 for i in range(data.shape[0])]
e4=[0 for i in range(data.shape[0])]
e5=[0 for i in range(data.shape[0])]
e6=[0 for i in range(data.shape[0])]
e7=[0 for i in range(data.shape[0])]
e8=[0 for i in range(data.shape[0])]
e9=[0 for i in range(data.shape[0])]
data['e1']=e1
data['e2']=e2
data['e3']=e3
data['e4']=e4
data['e5']=e5
data['e6']=e6
data['e7']=e7
data['e8']=e8
data['e9']=e9
# here we generating the neighbors of the target pixel

In [128]:
print(data.shape)

(28396, 16)


In [129]:
for index, row in data.iterrows():
    
    if row['frame']==0.0:
        pass
    else:
        current_u=row['u']
        current_v=row['v']
        previous_frame=row['frame']-1.0
        if current_u==0 and current_v==0.0 and  previous_frame==0.0:
            pass
        a0=data.loc[((abs(data['u']-current_u)==0.0) & (data['frame']==previous_frame) & (abs(data['v']-current_v)==0.0)) ]['e']
        a1=data.loc[((abs(data['u']==current_u -1))& (data['frame']==previous_frame) & (abs(data['v']==current_v -1))) ]['e']
        a2= data.loc[((abs(data['u']==current_u-1)) & (data['frame']==previous_frame) & (abs(data['v']-current_v)<1.0)) ]['e']
        a3= data.loc[((abs(data['u']==current_u-1)) & (data['frame']==previous_frame) & (abs(data['v']==current_v+1))) ]['e']
        a4=data.loc[((abs(data['u']-current_u)<1.0) & (data['frame']==previous_frame) & (abs(data['v']==current_v-1))) ]['e']
        a5= data.loc[((abs(data['u']-current_u)<1.0) & (data['frame']==previous_frame) & (abs(data['v']==current_v+1))) ]['e']
        a6=data.loc[((abs(data['u']==current_u+1)) & (data['frame']==previous_frame) & (abs(data['v']==current_v-1))) ]['e']
        a7= data.loc[((abs(data['u']==current_u+1)) & (data['frame']==previous_frame) & (abs(data['v']-current_v)<1.0)) ]['e']
        a8=data.loc[((abs(data['u']==current_u+1)) & (data['frame']==previous_frame) & (abs(data['v']==current_v+1))) ]['e']
        if (len(a0)!=0 and len(a0)<2) :   
            data.at[index,'e1'] = a0
        
        if (len(a1)!=0 and len(a1)<2):
            data.at[index,'e2'] = a1
        
        if (len(a2)!=0 and len(a2)<2):
            data.at[index,'e3'] = a2
           
        if (len(a3)!=0 and len(a3)<2):
            data.at[index,'e4'] = a3
        
        if (len(a4)!=0 and len(a4)<2):
            data.at[index,'e5'] = a4
        
        if (len(a5)!=0 and len(a5)<2):
            data.at[index,'e6'] = a5
        
        if (len(a6)!=0 and len(a6)<2):
            data.at[index,'e7'] = a6
        if (len(a7)!=0 and len(a7)<2):
            data.at[index,'e8'] = a7
       
        if (len(a8)!=0 and len(a8)<2):
            data.at[index,'e9'] = a8
# here we are filling the values of e_k for neighbor pixels       

In [130]:
data=data.drop(['e'],axis=1) 

In [131]:
print(data.shape)

(28396, 15)


In [132]:
target=data['y']# the target variable
data_input=data.drop(['y'], axis=1) #the training examples

In [133]:
data=pd.get_dummies(data, prefix=['frame'], columns=['frame'])#deal with categorical variable

In [134]:
# partitionnate the data into train and test
X_train, X_test, y_train, y_test = train_test_split(data_input, target, test_size=0.33, random_state=42)

In [135]:
clf = LogisticRegression(random_state=0).fit(X_train,y_train)# train the model using logisitc regression
predictions=clf.predict(X_test)
acc=accuracy_score(y_test, predictions, normalize=True)# determine the accuracy of the model
print(acc)

C:\Users\LENOVO\anaconda2021\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8213637818802689
